In [1]:
using ForneyLab, LinearAlgebra, Random, Plots

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1278
┌ Warning: Package ForneyLab does not have DataStructures in its dependencies:
│ - If you have ForneyLab checked out for development and have
│   added DataStructures as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ForneyLab
└ Loading DataStructures into ForneyLab from project dependency, future warnings for ForneyLab are suppressed.
  ** incremental compilation may be fatally broken for this module **



Model specification $p(y,z) = \mathcal{N}(y=13.5,\prod\limits_{i=1}^{3}x_i^{[z==i]},3) \mathcal{C}at(z;[0.49,0.01,0.5])$

where $x_1 = -10, x_2 = 10, x_3 = 20$

# Manual Calculation

In [2]:
loglikes = [logPdf(ProbabilityDistribution(Univariate, GaussianMeanVariance, m=-10, v=3),13.5),
logPdf(ProbabilityDistribution(Univariate, GaussianMeanVariance, m=10, v=3),13.5),
logPdf(ProbabilityDistribution(Univariate, GaussianMeanVariance, m=20, v=3),13.5)]

3-element Array{Float64,1}:
 -93.50991134420539
  -3.509911344205394
  -8.509911344205396

In [3]:
potential = ProbabilityDistribution(Univariate, Categorical, p=exp.(loglikes)/sum(exp.(loglikes)))

Cat(p=[8.14e-40, 0.99, 6.69e-03])


In [4]:
prior = ProbabilityDistribution(Univariate, Categorical, p=[0.49,0.01,0.5])

Cat(p=[0.49, 1.00e-02, 0.50])


In [5]:
posterior = prod!(potential,prior)

Cat(p=[7.53e-11, 0.75, 0.25])


In [6]:
# Symmetric KL divergence to measure performance
symmetric_KL(p,q) = (mean(p)'*log.(mean(p)) + mean(q)'*log.(mean(q)) - mean(q)'*log.(mean(p)) - mean(p)'*log.(mean(q)))/2

symmetric_KL (generic function with 1 method)

# EVMP

In [7]:
# Model
graph = FactorGraph()

f(z) = z[1]*-10. + z[2]*10. + z[3]*20.

@RV z ~ Categorical([0.49,0.01,0.5])

@RV s ~ Nonlinear{Sampling}(z,g=f)
@RV y ~ GaussianMeanVariance(s,3.)
placeholder(y,:y)
;

In [8]:
algo = messagePassingAlgorithm(z)
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code))
;

In [9]:
KL_EVMP = []
for i=1:100
    Random.seed!(i)
    data = Dict(:y => 13.5)
    marginals = Dict()

    step!(data, marginals)
    estimate = ProbabilityDistribution(Univariate, Categorical, p=mean(marginals[:z]))
    push!(KL_EVMP,symmetric_KL(estimate,posterior))
end

In [10]:
mean(KL_EVMP)

0.011763942391736025

In [11]:
var(KL_EVMP)

0.0006955651659333311

# AIS-MP

In [12]:
# Model
graph = FactorGraph()

f(z) = z[1]*-10. + z[2]*10. + z[3]*20.

@RV z ~ Categorical([0.49,0.01,0.5])

@RV s ~ Aismp(z,g=f)
@RV y ~ GaussianMeanVariance(s,3.)
placeholder(y,:y)
;

In [13]:
algo = messagePassingAlgorithm(z)
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code))
;

In [14]:
KL_AISMP = []
for i=1:100
    Random.seed!(i)
    data = Dict(:y => 13.5)
    marginals = Dict()

    step!(data, marginals)
    estimate = marginals[:z]
    push!(KL_AISMP,symmetric_KL(estimate,posterior))
end

In [15]:
mean(KL_AISMP)

0.006204354185200648

In [16]:
var(KL_AISMP)

1.9856147872346943e-5